# data prep


In [1]:
# imports
import re
import os
import json
import pprint
import datetime
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
# read in the training data set
census_train = pd.read_csv("C:/Users/EJE913/ct16_cap1_ds6/project_2/data/ad_placement/census-income.csv", header=None) 

In [3]:
# attach headers to the data
census_train.columns= ['age', 'class_of_worker', 'industry_recode','occupation_recode', 'education', 'wage_per_hour', \
                      'enroll_in_ed', 'marital', 'industry_cd', 'occupation_cd', 'race', 'hispanic_origin', 'sex', \
                      'labor_union', 'reason_unemployed', 'employ_stat', 'capital_gains', 'capital_losses', 'dividends', \
                      'tax_filer', 'region_prev_resid', 'state_prev_resid', 'household_stat','household_summary', 'weight',\
                      'change_in_msa', 'change_in_reg', 'move_within_reg', 'live_here_before', 'prev_resid_sunbelt',\
                      'num_person_worked', 'under_18', 'birth_country_dad', 'birth_country_mom', 'birth_country_self', \
                      'citizenship', 'owner_self_employed', 'veterans_admin', 'veterans_benefits', 'weekes_worked_year', 'year', \
                       'target']  

In [5]:
# read in the test dataset
census_train_test = pd.read_csv("C:/Users/EJE913/ct16_cap1_ds6/project_2/data/ad_placement/census-income-test.csv", header=None) 
census_train_test.columns= ['age', 'class_of_worker', 'industry_recode','occupation_recode', 'education', 'wage_per_hour', \
                      'enroll_in_ed', 'marital', 'industry_cd', 'occupation_cd', 'race', 'hispanic_origin', 'sex', \
                      'labor_union', 'reason_unemployed', 'employ_stat', 'capital_gains', 'capital_losses', 'dividends', \
                      'tax_filer', 'region_prev_resid', 'state_prev_resid', 'household_stat','household_summary', 'weight',\
                      'change_in_msa', 'change_in_reg', 'move_within_reg', 'live_here_before', 'prev_resid_sunbelt',\
                      'num_person_worked', 'under_18', 'birth_country_dad', 'birth_country_mom', 'birth_country_self', \
                      'citizenship', 'owner_self_employed', 'veterans_admin', 'veterans_benefits', 'weekes_worked_year', 'year', \
                       'target']  

# clean up columns, create buckets, change char to num etc.

In [8]:
# make target 0/1
census_train['target_bin']=census_train['target'].apply(lambda x: 1 if x.strip()=='50000+.' 
                                                        else 0 if x.strip()=='- 50000.' else 'miss')

# lets bucket ages 

# 0-15 (child-1), 16-22 (school-2), 23-39 (entry-3), 40-66 (manage-4), 67+ (retire-5)
census_train['age_bucket']=census_train['age'].apply(lambda x: 1 if 0<=x<=15 
                                                     else 2 if 16<=x<=22 
                                                     else 3 if 23<=x<=39 
                                                     else 4 if 40<=x<=66
                                                     else 5 if x>=67 
                                                     else 'miss')
census_train['age_bucket'].value_counts()

#class of worker
'''
All people over the age of 15 who have been employed at any 
time are asked to designate the type of work normally done or the work performed most regularly. 
'''

print census_train['class_of_worker'].value_counts()


 Not in universe                   100245
 Private                            72028
 Self-employed-not incorporated      8445
 Local government                    7784
 State government                    4227
 Self-employed-incorporated          3265
 Federal government                  2925
 Never worked                         439
 Without pay                          165
Name: class_of_worker, dtype: int64


In [10]:
# convert to numebrs
census_train['class_code']=census_train['class_of_worker'].apply(lambda x: 0 if x.strip()=='Not in universe' 
                                                                else 1 if x.strip()=='Private'
                                                                else 2 if x.strip()=='Self-employed-not incorporated'
                                                                else 3 if x.strip()=='Local government'
                                                                else 4 if x.strip()=='State government'
                                                                else 5 if x.strip()=='Federal government'
                                                                else 6 if x.strip()=='Never worked'
                                                                else 7 if x.strip()=='Without pay'
                                                                else 8 if x.strip()=='Self-employed-incorporated'
                                                                else 'miss')
census_train['class_code'].value_counts()

0    100245
1     72028
2      8445
3      7784
4      4227
8      3265
5      2925
6       439
7       165
Name: class_code, dtype: int64

In [13]:
'''
0: not in universe
1: agriculture, fishing, forestry, hunting
2: mining
3: construction
4: manufacturing
5: wholesale and retail trade
6: transportation and utilities
7: information
8: financial activities
9: professional and business services
10: education and health services
11: leisure and hospitality
12: other services
13: public administration
14: armed forces
'''

industry_major={0:0,1:1,2:1,3:2,4:3,5:4,6:4,7:4,8:4,9:4,10:4,11:4,12:4,13:4,14:4,15:4,16:4,17:4,18:4,19:4,20:4,21:4,
               22:4,23:4,24:4,25:4,26:4,27:4,28:4,29:6,30:6,31:6,32:5,33:5,34:8,35:8,36:12,37:9,38:9,39:12,40:11,41:10,
               42:10,43:10,44:12,45:9,46:1,47:13,48:13,49:13,50:13,51:14}
census_train['industry_major_cd']=census_train['industry_recode'].apply(lambda x: industry_major.get(x))
census_train['industry_major_cd'].value_counts()

0     100684
5      20666
10     16930
4      15912
9      10133
6       6568
12      6431
8       6145
3       5984
13      4610
1       3210
11      1651
2        563
14        36
Name: industry_major_cd, dtype: int64

In [16]:
#lets group these a bit and convert to numbers
census_train['education_bucket']=census_train['education'].apply(lambda x: 2 if x.strip()=='Less than 1st grade'
                                                                else 2 if x.strip()=='1st 2nd 3rd or 4th grade'
                                                                else 3 if x.strip()=='5th or 6th grade'
                                                                else 1 if x.strip()=='Children'
                                                                else 3 if x.strip()=='7th and 8th grade'
                                                                else 4 if x.strip()=='9th grade'
                                                                else 4 if x.strip()=='10th grade'
                                                                else 4 if x.strip()=='11th grade'
                                                                else 4 if x.strip()=='12th grade no diploma'
                                                                else 5 if x.strip()=='High school graduate'
                                                                else 6 if x.strip()=='Some college but no degree'
                                                                else 6 if x.strip()=='Associates degree-occup /vocational'
                                                                else 6 if x.strip()=='Associates degree-academic program'
                                                                else 7 if x.strip()=='Bachelors degree(BA AB BS)'
                                                                else 8 if x.strip()=='Masters degree(MA MS MEng MEd MSW MBA)'
                                                                else 8 if x.strip()=='Prof school degree (MD DDS DVM LLB JD)'
                                                                else 8 if x.strip()=='Doctorate degree(PhD EdD)'
                                                                else 'miss')

In [19]:
#this seems pretty useless to me since 94% have the same value so let's drop it!
census_train.drop(['enroll_in_ed'], axis=1, inplace=True)

# marital
census_train['marital'].value_counts()

 Never married                      86485
 Married-civilian spouse present    84222
 Divorced                           12710
 Widowed                            10463
 Separated                           3460
 Married-spouse absent               1518
 Married-A F spouse present           665
Name: marital, dtype: int64

In [20]:
#convert to number
census_train['marital_code']=census_train['marital'].apply(lambda x: 0 if x.strip()=='Never married' 
                                                                else 1 if x.strip()=='Married-civilian spouse present'
                                                                else 2 if x.strip()=='Divorced'
                                                                else 3 if x.strip()=='Widowed'
                                                                else 4 if x.strip()=='Separated'
                                                                else 5 if x.strip()=='Married-spouse absent'
                                                                else 6 if x.strip()=='Married-A F spouse present'
                                                                else 'miss')



In [24]:
census_train['tgt_bin'] = [1 if x.strip() == '- 50000.' else 0 if x.strip() == '50000+.' else -1 for x in census_train.target]
census_train.tgt_bin.value_counts()

1    187141
0     12382
Name: tgt_bin, dtype: int64

In [25]:
print census_train.drop_duplicates().shape
#census_train.dtypes

(196291, 48)


In [28]:
#'birth_country_dad_cd' - US/Americas/Europe/Asia/?
#'birth_country_mom_cd' - US/Americas/Europe/Asia/?
#'birth_country_self_cd' - US/Americas/Europe/Asia/?
# 'year' - 94/95 (Don't use)
# 'citizenship' - native vs foreign
# 'veterans_admin' - 99% not in universe (Don't use)
# 'veterans_benefits' - 2 (150130)/ 0 (47409)/ 1(1984)
# 'owner_self_employed - 0 (180672)/ 2 (16153)/ 1 (2698) - 91% have 0 not owner neither self-employed
# 'weekes_worked_year' - binned as 0/ 1-48/ 49-52

US = ['United-States', 'Puerto-Rico', 'Canada', 'Outlying-U S (Guam USVI etc)']
Americas = ['Mexico', 'Guatemala', 'Honduras', 'Nicaragua', 'Panama', 'El-Salvador', \
            'Cuba', 'Haiti', 'Dominican-Republic', 'Jamaica', 'Trinadad&Tobago', \
            'Columbia', 'Ecuador', 'Peru']
Europe = ['Germany', 'England', 'Ireland', 'Italy', 'Portugal', 'Hungary', 'Greece', 'Poland', 'Scotland', 'France', \
          'Holand-Netherlands',  'Yugoslavia']
Asias = ['Philippines', 'Vietnam', 'Japan', 'China', 'South Korea', 'India', 'Taiwan',  'Iran', 'Cambodia', \
         'Laos', 'Thailand', 'Hong Kong']

census_train['birth_country_mom_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_train.birth_country_mom]

census_train['birth_country_dad_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_train.birth_country_dad]

census_train['parents_country_cd'] = ["US" if x == "US" and y == "US" else "US-Foreign" \
                                      if (x == "US" and y != "US") or (x != "US" and y == "US") else "Foreign"\
                                      for x, y in zip(census_train.birth_country_mom_cd, census_train.birth_country_dad_cd)]

census_train['birth_country_self_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_train.birth_country_self]
census_train['citizenship_cd'] = ['Native' if x.strip()[:6] == 'Native' else 'Foreign' if x.strip()[:7] == 'Foreign'\
                                  else '?' for x in census_train.citizenship]
census_train['weeks_worked_bin'] = pd.Series([0 if x == 0 else 1 if x <= 48 else 2 for x in census_train.weekes_worked_year])

ft_proxy = [40 if x.strip() in ["Full-time schedules", "PT for econ reasons usually FT", \
            "PT for non-econ reasons usually FT", "Children or Armed Forces"] else 20 for x in census_train.employ_stat]
census_train['work_time_proxy'] = [x*y for x, y in zip(ft_proxy, census_train.weekes_worked_year)]

census_train.work_time_proxy.describe()
# birth_country_dad_cd, parents_country_cd, citizenship_cd, weeks_worked_bin

count    199523.000000
mean        903.951224
std         969.402343
min           0.000000
25%           0.000000
50%         240.000000
75%        2080.000000
max        2080.000000
Name: work_time_proxy, dtype: float64

In [29]:
census_train[census_train.birth_country_mom_cd == "?"].birth_country_mom.value_counts()

 ?    6119
Name: birth_country_mom, dtype: int64

In [30]:
census_train[census_train.birth_country_dad_cd == "?"].birth_country_dad.value_counts()

 ?    6713
Name: birth_country_dad, dtype: int64

In [31]:
census_train['parents_country_cd'] = ["US" if x == "US" and y == "US" else "US-Foreign" \
                                      if (x == "US" and y != "US") or (x != "US" and y == "US") else "Foreign"\
                                      for x, y in zip(census_train.birth_country_mom_cd, census_train.birth_country_dad_cd)]

In [32]:
print pd.crosstab(census_train.birth_country_mom_cd, census_train.birth_country_dad_cd)

birth_country_dad_cd     ?     A     Am     E      US
birth_country_mom_cd                                 
?                     4879    69    102   243     826
A                       58  4476     43    20     488
Am                     171    60  14692    78    1341
E                      271    58     52  5313    1723
US                    1334   301   1750  2171  159004


In [33]:
census_train['birth_country_self_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_train.birth_country_self]
census_train.birth_country_self_cd.value_counts()

US    179208
Am     10291
A       3703
?       3393
E       2928
Name: birth_country_self_cd, dtype: int64

In [34]:
census_train['citizenship_cd'] = ['Native' if x.strip()[:6] == 'Native' else 'Foreign' if x.strip()[:7] == 'Foreign'\
                                  else '?' for x in census_train.citizenship]

In [35]:
census_train.owner_self_employed.value_counts()/len(census_train)

0    0.905520
2    0.080958
1    0.013522
Name: owner_self_employed, dtype: float64

In [36]:
census_train.veterans_admin.value_counts()/len(census_train)

 Not in universe    0.990056
 No                 0.007984
 Yes                0.001960
Name: veterans_admin, dtype: float64

In [37]:
census_train.veterans_benefits.value_counts()/len(census_train)

2    0.752445
0    0.237612
1    0.009944
Name: veterans_benefits, dtype: float64

In [38]:
census_train['weeks_worked_bin'] = pd.Series([0 if x == 0 else 1 if x <= 48 else 2 for x in census_train.weekes_worked_year])

In [42]:
census_train['occupation_cd'] = (census_train['occupation_cd']).apply(func=str.strip)
census_train['target'] = (census_train['target']).apply(func=str.strip)

In [43]:
# Creating a duplicate column to convert occupation_cd_num to numbers
census_train['occupation_cd_num']= census_train['occupation_cd']

In [44]:
'''Converting occupation_cd to numbers:

'Not in universe': 0
'Precision production craft & repair': 1
'Professional specialty': 2
'Executive admin and managerial': 3
'Handlers equip cleaners etc':4 
'Adm support including clerical':5
'Machine operators assmblrs & inspctrs':6 
'Other service':7 
'Sales':8
'Private household services':9 
'Technicians and related support':10
'Transportation and material moving':11
'Farming forestry and fishing':12
'Protective services':13
'Armed Forces':14'''


census_train['occupation_cd_num'] = census_train['occupation_cd_num'].replace(['Not in universe', 'Precision production craft & repair',
       'Professional specialty', 'Executive admin and managerial',
       'Handlers equip cleaners etc', 'Adm support including clerical',
       'Machine operators assmblrs & inspctrs', 'Other service', 'Sales',
       'Private household services', 'Technicians and related support',
       'Transportation and material moving',
       'Farming forestry and fishing', 'Protective services',
       'Armed Forces'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])


del census_train['race'] 
del census_train['hispanic_origin']
del census_train['sex']
del census_train['labor_union']
del census_train['reason_unemployed'] 
del census_train['employ_stat']

In [45]:
#Capital Gains to Household Summary
census_train_reformatted = census_train

#Capital_Gains & Capital_Losses & Dividends
#census_train_reformatted.dividends = census_train_reformatted.dividends.str.strip()
census_train_reformatted["net_capital_gains"] = census_train_reformatted.capital_gains - census_train_reformatted.capital_losses + census_train_reformatted.dividends 
del census_train_reformatted['capital_gains']
del census_train_reformatted['capital_losses']
del census_train_reformatted['dividends']

#Tax_filer
census_train_reformatted.tax_filer = census_train_reformatted.tax_filer.str.strip()

census_train_reformatted.tax_filer.replace('Nonfiler',0, inplace=True)
census_train_reformatted.tax_filer.replace('Joint both under 65',1, inplace=True)
census_train_reformatted.tax_filer.replace('Single',1, inplace=True)
census_train_reformatted.tax_filer.replace('Joint both 65+',1, inplace=True)
census_train_reformatted.tax_filer.replace('Head of household',1, inplace=True)
census_train_reformatted.tax_filer.replace('Joint one under 65 & one 65+',1, inplace=True)

#Region_prev_resid
census_train_reformatted.region_prev_resid = census_train_reformatted.region_prev_resid.str.strip()

census_train_reformatted.region_prev_resid.replace('Not in universe',0, inplace=True)
census_train_reformatted.region_prev_resid.replace('South',1, inplace=True)
census_train_reformatted.region_prev_resid.replace('West',2, inplace=True)
census_train_reformatted.region_prev_resid.replace('Midwest',3, inplace=True)
census_train_reformatted.region_prev_resid.replace('Northeast',4, inplace=True)
census_train_reformatted.region_prev_resid.replace('Abroad',5, inplace=True)

#State_prev_resid
del census_train_reformatted['state_prev_resid']

#household_stat
del census_train_reformatted['household_stat']

#household_summary
census_train_reformatted.household_summary = census_train_reformatted.household_summary.str.strip()

census_train_reformatted.household_summary.replace('Householder',1, inplace=True)
census_train_reformatted.household_summary.replace('Child under 18 never married',0, inplace=True)
census_train_reformatted.household_summary.replace('Spouse of householder',0, inplace=True)
census_train_reformatted.household_summary.replace('Child 18 or older',0, inplace=True)
census_train_reformatted.household_summary.replace('Other relative of householder',0, inplace=True)
census_train_reformatted.household_summary.replace('Nonrelative of householder',0, inplace=True)
census_train_reformatted.household_summary.replace('Group Quarters- Secondary individual',0, inplace=True)
census_train_reformatted.household_summary.replace('Child under 18 ever married',0, inplace=True)

census_train_reformatted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 46 columns):
age                      199523 non-null int64
class_of_worker          199523 non-null object
industry_recode          199523 non-null int64
occupation_recode        199523 non-null int64
education                199523 non-null object
wage_per_hour            199523 non-null int64
marital                  199523 non-null object
industry_cd              199523 non-null object
occupation_cd            199523 non-null object
tax_filer                199523 non-null int64
region_prev_resid        199523 non-null int64
household_summary        199523 non-null int64
weight                   199523 non-null float64
change_in_msa            199523 non-null object
change_in_reg            199523 non-null object
move_within_reg          199523 non-null object
live_here_before         199523 non-null object
prev_resid_sunbelt       199523 non-null object
num_person_worked        199523

In [51]:
subname = ['change_in_msa', 'change_in_reg', 'live_here_before', 'prev_resid_sunbelt',
              'num_person_worked', 'under_18'] 

#new_train = pd.get_dummies(census_train,columns=subname) 

In [53]:
subname

['change_in_msa',
 'change_in_reg',
 'live_here_before',
 'prev_resid_sunbelt',
 'num_person_worked',
 'under_18']

In [59]:
census_train.drop(['age', 'birth_country_dad', 'birth_country_mom', 'birth_country_self', 'birth_country_self_cd', 
                   'citizenship','class_of_worker', 'education', 'industry_cd', 'industry_recode', 'marital', 'move_within_reg'
                   , 'occupation_cd', 'occupation_recode', 'owner_self_employed', 'tgt_bin', 'target', 'veterans_admin', 
                   'veterans_benefits', 'weekes_worked_year', 'year'], axis=1, inplace=True)

In [60]:
census_train.head()

,wage_per_hour,tax_filer,region_prev_resid,household_summary,weight,change_in_msa,change_in_reg,live_here_before,prev_resid_sunbelt,num_person_worked,...,education_bucket,marital_code,birth_country_mom_cd,birth_country_dad_cd,parents_country_cd,citizenship_cd,weeks_worked_bin,work_time_proxy,occupation_cd_num,net_capital_gains
0,0,0,0,0,1700.09,?,?,Not in universe under 1 year old,?,0,...,5,3,US,US,US,Native,0,0,0,0
1,0,1,1,1,1053.55,MSA to MSA,Same county,No,Yes,1,...,6,2,US,US,US,Native,2,2080,1,0
2,0,0,0,0,991.95,?,?,Not in universe under 1 year old,?,0,...,4,0,A,A,Foreign,Foreign,0,0,0,0
3,0,0,0,0,1758.14,Nonmover,Nonmover,Yes,Not in universe,0,...,1,0,US,US,US,Native,0,0,0,0
4,0,0,0,0,1069.16,Nonmover,Nonmover,Yes,Not in universe,0,...,1,0,US,US,US,Native,0,0,0,0


In [61]:
dummies=['age_bucket', 'birth_country_dad_cd', 'birth_country_mom_cd', 'change_in_msa', 
         'change_in_reg', 'citizenship_cd', 'class_code', 'education_bucket', 'industry_major_cd', 
         'live_here_before', 'marital_code', 'occupation_cd_num', 'parents_country_cd', 'prev_resid_sunbelt', 
         'region_prev_resid', 'under_18', 'weeks_worked_bin']

In [63]:
census_train_final=pd.get_dummies(census_train, columns=dummies)

In [67]:
fiftyk_features=[]
for index in census_train_final:
    fiftyk_features.append(index)
    

In [76]:
fiftyk_features.remove('weight')

# knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#define a list of features as it becomes easier to reuse it 
#fiftyk_features=[]
#for index in census_train:
#    fiftyk_features.append(index)

#define X(features) and Y(target) vectors for KNeighborsClassifier model using training data
X_knn= census_train_final[fiftyk_features]
Y_knn= census_train_final['target_bin']

def fiftyk_knn_model(k_value): 
    #Create an object of KNeighborsClassifier class(calling the consturctor here)
    knn_model = KNeighborsClassifier(n_neighbors=k_value)
    print k_value

    #Calling .fit function on knn_model object to train the model using training data
    knn_model.fit(X_knn, Y_knn) 

    #Calling .predict function on the trained model(knn_model) for test data
    fiftyk_knn_pred = knn_model.predict(X_test[fiftyk_features])

    #Compute model accuracy
    model_accuracy= accuracy_score(y_test['target_bin'],fiftyk_knn_pred)

    return model_accuracy



#For loop to find the highest accuracy for range of k(number of neighbors) values

def fiftyk_knn_accuracy(k_limit):
    fiftyk_k_permutations = {}
    for k in range(1,k_limit+1):
        fiftyk_k_permutations.update({k:fiftyk_knn_model(k)})
    return fiftyk_k_permutations

print('K values (1 to 20) and corresponding accuracy:\n')
print(fiftyk_knn_accuracy(20))

print('\n K value with highest accuracy:')
highest = max(fiftyk_knn_accuracy(20), key=(fiftyk_knn_accuracy(20)).get) 
print(highest, fiftyk_knn_accuracy(20)[highest])



K values (1 to 20) and corresponding accuracy:

1


# regression

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score


#define X(features) and Y(target) vectors for LogisticRegression model using training data
X_logreg= census_train_final[fiftyk_features]
Y_logreg= census_train_final['target_bin]

#Create an object of LogisticRegression class(calling the constructor here)
logreg_model = LogisticRegression()

#Calling .fit function on knn_model object to train the model using training data
logreg_model.fit(X_logreg, Y_logreg) 
    
#Calling .predict function on the trained model(knn_model) for test data
fifty_logreg_pred = logreg_model.predict(census_test[fiftyk_features])

#Compute model accuracy
logreg_accuracy= accuracy_score(fiftyk_test['target_bin],fifty_logreg_pred)

print('Accuracy for logistic regression model:')
print(logreg_accuracy)



In [74]:
census_test = pd.read_csv("C:/Users/eje913/ct16_cap1_ds6/project_2/data/ad_placement/census-income-test.csv", header=None) 
census_test.columns= ['age', 'class_of_worker', 'industry_recode','occupation_recode', 'education', 'wage_per_hour', \
                      'enroll_in_ed', 'marital', 'industry_cd', 'occupation_cd', 'race', 'hispanic_origin', 'sex', \
                      'labor_union', 'reason_unemployed', 'employ_stat', 'capital_gains', 'capital_losses', 'dividends', \
                      'tax_filer', 'region_prev_resid', 'state_prev_resid', 'household_stat','household_summary', 'weight',\
                      'change_in_msa', 'change_in_reg', 'move_within_reg', 'live_here_before', 'prev_resid_sunbelt',\
                      'num_person_worked', 'under_18', 'birth_country_dad', 'birth_country_mom', 'birth_country_self', \
                      'citizenship', 'owner_self_employed', 'veterans_admin', 'veterans_benefits', 'weekes_worked_year', 'year', \
                       'target']  
# make target 0/1
census_test['target_bin']=census_test['target'].apply(lambda x: 1 if x.strip()=='50000+.' else 0 if x.strip()=='- 50000.' else 'miss')
#print census_train.groupby(['target_bin']).count()

# lets bucket ages 

# 0-15 (child-1), 16-22 (school-2), 23-39 (entry-3), 40-66 (manage-4), 67+ (retire-5)
census_test['age_bucket']=census_test['age'].apply(lambda x: 1 if 0<=x<=15 else 2 if 16<=x<=22 else 3 if 23<=x<=39 else 4 if 40<=x<=66
                                             else 5 if x>=67 else 'miss')
#class of worker
'''
All people over the age of 15 who have been employed at any 
time are asked to designate the type of work normally done or the work performed most regularly. 
'''
# convert to numebrs
census_test['class_code']=census_test['class_of_worker'].apply(lambda x: 0 if x.strip()=='Not in universe' 
                                                                else 1 if x.strip()=='Private'
                                                                else 2 if x.strip()=='Self-employed-not incorporated'
                                                                else 3 if x.strip()=='Local government'
                                                                else 4 if x.strip()=='State government'
                                                                else 5 if x.strip()=='Federal government'
                                                                else 6 if x.strip()=='Never worked'
                                                                else 7 if x.strip()=='Without pay'
                                                                else 8 if x.strip()=='Self-employed-incorporated'
                                                                else 'miss')
'''
0: not in universe
1: agriculture, fishing, forestry, hunting
2: mining
3: construction
4: manufacturing
5: wholesale and retail trade
6: transportation and utilities
7: information
8: financial activities
9: professional and business services
10: education and health services
11: leisure and hospitality
12: other services
13: public administration
14: armed forces
'''

industry_major={0:0,1:1,2:1,3:2,4:3,5:4,6:4,7:4,8:4,9:4,10:4,11:4,12:4,13:4,14:4,15:4,16:4,17:4,18:4,19:4,20:4,21:4,
               22:4,23:4,24:4,25:4,26:4,27:4,28:4,29:6,30:6,31:6,32:5,33:5,34:8,35:8,36:12,37:9,38:9,39:12,40:11,41:10,
               42:10,43:10,44:12,45:9,46:1,47:13,48:13,49:13,50:13,51:14}

census_test['industry_major_cd']=census_test['industry_recode'].apply(lambda x: industry_major.get(x))

#lets group these a bit and convert to numbers
census_test['education_bucket']=census_test['education'].apply(lambda x: 2 if x.strip()=='Less than 1st grade'
                                                                else 2 if x.strip()=='1st 2nd 3rd or 4th grade'
                                                                else 3 if x.strip()=='5th or 6th grade'
                                                                else 1 if x.strip()=='Children'
                                                                else 3 if x.strip()=='7th and 8th grade'
                                                                else 4 if x.strip()=='9th grade'
                                                                else 4 if x.strip()=='10th grade'
                                                                else 4 if x.strip()=='11th grade'
                                                                else 4 if x.strip()=='12th grade no diploma'
                                                                else 5 if x.strip()=='High school graduate'
                                                                else 6 if x.strip()=='Some college but no degree'
                                                                else 6 if x.strip()=='Associates degree-occup /vocational'
                                                                else 6 if x.strip()=='Associates degree-academic program'
                                                                else 7 if x.strip()=='Bachelors degree(BA AB BS)'
                                                                else 8 if x.strip()=='Masters degree(MA MS MEng MEd MSW MBA)'
                                                                else 8 if x.strip()=='Prof school degree (MD DDS DVM LLB JD)'
                                                                else 8 if x.strip()=='Doctorate degree(PhD EdD)'
                                                                else 'miss')


#this seems pretty useless to me since 94% have the same value so let's drop it!
census_test.drop(['enroll_in_ed'], axis=1, inplace=True)

#convert to number
census_test['marital_code']=census_test['marital'].apply(lambda x: 0 if x.strip()=='Never married' 
                                                                else 1 if x.strip()=='Married-civilian spouse present'
                                                                else 2 if x.strip()=='Divorced'
                                                                else 3 if x.strip()=='Widowed'
                                                                else 4 if x.strip()=='Separated'
                                                                else 5 if x.strip()=='Married-spouse absent'
                                                                else 6 if x.strip()=='Married-A F spouse present'
                                                                else 'miss')

#'birth_country_dad_cd' - US/Americas/Europe/Asia/?
#'birth_country_mom_cd' - US/Americas/Europe/Asia/?
#'birth_country_self_cd' - US/Americas/Europe/Asia/?
# 'year' - 94/95 (Don't use)
# 'citizenship' - native vs foreign
# 'veterans_admin' - 99% not in universe (Don't use)
# 'veterans_benefits' - 2 (150130)/ 0 (47409)/ 1(1984)
# 'owner_self_employed - 0 (180672)/ 2 (16153)/ 1 (2698) - 91% have 0 not owner neither self-employed
# 'weekes_worked_year' - binned as 0/ 1-48/ 49-52

US = ['United-States', 'Puerto-Rico', 'Canada', 'Outlying-U S (Guam USVI etc)']
Americas = ['Mexico', 'Guatemala', 'Honduras', 'Nicaragua', 'Panama', 'El-Salvador', \
            'Cuba', 'Haiti', 'Dominican-Republic', 'Jamaica', 'Trinadad&Tobago', \
            'Columbia', 'Ecuador', 'Peru']
Europe = ['Germany', 'England', 'Ireland', 'Italy', 'Portugal', 'Hungary', 'Greece', 'Poland', 'Scotland', 'France', \
          'Holand-Netherlands',  'Yugoslavia']
Asias = ['Philippines', 'Vietnam', 'Japan', 'China', 'South Korea', 'India', 'Taiwan',  'Iran', 'Cambodia', \
         'Laos', 'Thailand', 'Hong Kong']

census_test['birth_country_mom_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_test.birth_country_mom]

census_test['birth_country_dad_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_test.birth_country_dad]

census_test['parents_country_cd'] = ["US" if x == "US" and y == "US" else "US-Foreign" \
                                      if (x == "US" and y != "US") or (x != "US" and y == "US") else "Foreign"\
                                      for x, y in zip(census_test.birth_country_mom_cd, census_test.birth_country_dad_cd)]

census_test['birth_country_self_cd'] = ['US' if x.strip() in US else 'Am' if x.strip() in Americas \
                                        else 'E' if x.strip() in Europe else 'A' if x.strip() in Asias \
                                        else '?' for x in census_test.birth_country_self]
census_test['citizenship_cd'] = ['Native' if x.strip()[:6] == 'Native' else 'Foreign' if x.strip()[:7] == 'Foreign'\
                                  else '?' for x in census_test.citizenship]
census_test['weeks_worked_bin'] = pd.Series([0 if x == 0 else 1 if x <= 48 else 2 for x in census_test.weekes_worked_year])

ft_proxy = [40 if x.strip() in ["Full-time schedules", "PT for econ reasons usually FT", \
            "PT for non-econ reasons usually FT", "Children or Armed Forces"] else 20 for x in census_test.employ_stat]
census_test['work_time_proxy'] = [x*y for x, y in zip(ft_proxy, census_test.weekes_worked_year)]

# birth_country_dad_cd, parents_country_cd, citizenship_cd, weeks_worked_bin

'''Converting occupation_cd to numbers:

'Not in universe': 0
'Precision production craft & repair': 1
'Professional specialty': 2
'Executive admin and managerial': 3
'Handlers equip cleaners etc':4 
'Adm support including clerical':5
'Machine operators assmblrs & inspctrs':6 
'Other service':7 
'Sales':8
'Private household services':9 
'Technicians and related support':10
'Transportation and material moving':11
'Farming forestry and fishing':12
'Protective services':13
'Armed Forces':14'''

census_test['occupation_cd'] = (census_test['occupation_cd']).apply(func=str.strip)
census_test['occupation_cd_num']= census_test['occupation_cd']

census_test['occupation_cd_num'] = census_test['occupation_cd_num'].replace(['Not in universe', 'Precision production craft & repair',
       'Professional specialty', 'Executive admin and managerial',
       'Handlers equip cleaners etc', 'Adm support including clerical',
       'Machine operators assmblrs & inspctrs', 'Other service', 'Sales',
       'Private household services', 'Technicians and related support',
       'Transportation and material moving',
       'Farming forestry and fishing', 'Protective services',
       'Armed Forces'], [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])


del census_test['race'] 
del census_test['hispanic_origin']
del census_test['sex']
del census_test['labor_union']
del census_test['reason_unemployed'] 
del census_test['employ_stat']

#Capital Gains to Household Summary
census_test_reformatted = census_test

#Capital_Gains & Capital_Losses & Dividends
#census_train_reformatted.dividends = census_train_reformatted.dividends.str.strip()
census_test_reformatted["net_capital_gains"] = census_test_reformatted.capital_gains - census_test_reformatted.capital_losses + census_test_reformatted.dividends 
del census_test_reformatted['capital_gains']
del census_test_reformatted['capital_losses']
del census_test_reformatted['dividends']

#Tax_filer
census_test_reformatted.tax_filer = census_test_reformatted.tax_filer.str.strip()

census_test_reformatted.tax_filer.replace('Nonfiler',0, inplace=True)
census_test_reformatted.tax_filer.replace('Joint both under 65',1, inplace=True)
census_test_reformatted.tax_filer.replace('Single',1, inplace=True)
census_test_reformatted.tax_filer.replace('Joint both 65+',1, inplace=True)
census_test_reformatted.tax_filer.replace('Head of household',1, inplace=True)
census_test_reformatted.tax_filer.replace('Joint one under 65 & one 65+',1, inplace=True)

#Region_prev_resid
census_test_reformatted.region_prev_resid = census_test_reformatted.region_prev_resid.str.strip()

census_test_reformatted.region_prev_resid.replace('Not in universe',0, inplace=True)
census_test_reformatted.region_prev_resid.replace('South',1, inplace=True)
census_test_reformatted.region_prev_resid.replace('West',2, inplace=True)
census_test_reformatted.region_prev_resid.replace('Midwest',3, inplace=True)
census_test_reformatted.region_prev_resid.replace('Northeast',4, inplace=True)
census_test_reformatted.region_prev_resid.replace('Abroad',5, inplace=True)

#State_prev_resid
del census_test_reformatted['state_prev_resid']

#household_stat
del census_test_reformatted['household_stat']

#household_summary
census_test_reformatted.household_summary = census_test_reformatted.household_summary.str.strip()

census_test_reformatted.household_summary.replace('Householder',1, inplace=True)
census_test_reformatted.household_summary.replace('Child under 18 never married',0, inplace=True)
census_test_reformatted.household_summary.replace('Spouse of householder',0, inplace=True)
census_test_reformatted.household_summary.replace('Child 18 or older',0, inplace=True)
census_test_reformatted.household_summary.replace('Other relative of householder',0, inplace=True)
census_test_reformatted.household_summary.replace('Nonrelative of householder',0, inplace=True)
census_test_reformatted.household_summary.replace('Group Quarters- Secondary individual',0, inplace=True)
census_test_reformatted.household_summary.replace('Child under 18 ever married',0, inplace=True)

#census_test_reformatted.info()
census_test = census_test_reformatted

#drop: age, birth_country_dad, birth_country_mom, birth_country_self, birth_country_self_cd, citizenship,class_of_worker, 
# education, industry_code, industry_recode, marital,
# move_within_reg, occupation_cd, occupation_recode, owner_self_employed, tgt_bin, target, veterans_admin, veterans_benefits,
# weeks_worked_year, year

keep_vars = ['age_bucket', 'birth_country_dad_cd', 'birth_country_mom_cd', 'change_in_msa', 'change_in_reg', 'citizenship_cd',
             'class_code', 'education_bucket', 'industry_major_cd', 'live_here_before', 'marital_code', 'occupation_cd_num', \
             'parents_country_cd', 'prev_resid_sunbelt', 'region_prev_resid', 'under_18', 'weeks_worked_bin',\
             'net_capital_gains','num_person_worked','tax_filer', 'wage_per_hour', 'household_summary', 'work_time_proxy']

y_test = census_test['target_bin']
wgt_test = census_test['weight']

cat_vars = keep_vars[:17]
X_test = census_test[keep_vars]

X_test = pd.get_dummies(X_test, columns=cat_vars)